In [1]:
import os
import torch
import numpy as np
import pandas as pd
import sys
import random
import h5py
import torch
import torchvision
import ast
import cv2 
from torch.utils.tensorboard import SummaryWriter

from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from torch.autograd import Variable

sys.path.append('../')

from models.mcnnpytorch.src.utils import save_density_map
from models.mcnnpytorch.src import *
from models.mcnnpytorch.src.crowd_count import *
from models.mcnnpytorch.src.network import *
from models.mcnnpytorch.src.data_loader import ImageDataLoader
from models.mcnnpytorch.src.timer import *
from models.mcnnpytorch.src.evaluate_model import *
from models.mcnnpytorch.src import utils

torch.manual_seed(0)
np.random.seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [2]:
# Check to see if device can be trained on GPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

print(device)

cuda


In [3]:
# Cuda configurations

torch.cuda.empty_cache()
print(torch.cuda.memory_summary(device=None, abbreviated=False))

current_device = torch.cuda.current_device()
current_device_name = torch.cuda.get_device_name(current_device)
memory_allocated = torch.cuda.memory_allocated()
memory_cached = torch.cuda.memory_reserved()

print(
    f'Using gpu {current_device_name} with device number {current_device}.\n'
    f'Memory allocated = {memory_allocated}\n'
    f'Memory cached = {memory_cached}'
)

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |

In [4]:
try:
    from termcolor import cprint
except ImportError:
    cprint = None


def log_print(text, color=None, on_color=None, attrs=None):
    if cprint is not None:
        cprint(text, color=color, on_color=on_color, attrs=attrs)
    else:
        print(text)

In [5]:
# Directory Configurations

method = 'mcnn'
dataset_name = 'JHU'
output_dir = f'../output/{method}/saved_models/{dataset_name}/rerun'

# Training data path
train_path = '../data/JHU/train/consolidated'
train_gt_path = '../data/JHU/train/gt'

# Validation data path
val_path = '../data/JHU/val/consolidated'
val_gt_path = '../data/JHU/val/gt'

In [6]:
# Create output directory if it doesnt exist

if not os.path.exists(output_dir):
    os.mkdir(output_dir)

In [7]:
# load model

is_cuda = True  # Determine if we should use the CPU to train or GPU

model = CrowdCounter(is_cuda=is_cuda)  # is_cuda determines if all the input tensors should be converted to cuda tensors
network.weights_normal_init(model, dev=0.01)
model.train()

/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


CrowdCounter(
  (model): MCNN(
    (branch1): Sequential(
      (0): Conv2d(
        (conv): Conv2d(3, 16, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
        (relu): ReLU(inplace=True)
      )
      (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (2): Conv2d(
        (conv): Conv2d(16, 32, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
        (relu): ReLU(inplace=True)
      )
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (4): Conv2d(
        (conv): Conv2d(32, 16, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
        (relu): ReLU(inplace=True)
      )
      (5): Conv2d(
        (conv): Conv2d(16, 8, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
        (relu): ReLU(inplace=True)
      )
    )
    (branch2): Sequential(
      (0): Conv2d(
        (conv): Conv2d(3, 20, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
        (relu): ReLU(inplace=True)
      )
      (1): MaxPool2d

In [8]:
if is_cuda and torch.cuda.is_available():
    print("Changing to cuda weights")
    model.cuda()
    
# Model parameters

for name, param in model.named_parameters():
    print(f'{name}\t{param.device}\t{param.shape}')

# Print model's state_dict
print("\nModel's state_dict: \n")
for k, v in model.state_dict().items():
    print(k, "\t", v.dtype)

Changing to cuda weights
model.branch1.0.conv.weight	cuda:0	torch.Size([16, 3, 9, 9])
model.branch1.0.conv.bias	cuda:0	torch.Size([16])
model.branch1.2.conv.weight	cuda:0	torch.Size([32, 16, 7, 7])
model.branch1.2.conv.bias	cuda:0	torch.Size([32])
model.branch1.4.conv.weight	cuda:0	torch.Size([16, 32, 7, 7])
model.branch1.4.conv.bias	cuda:0	torch.Size([16])
model.branch1.5.conv.weight	cuda:0	torch.Size([8, 16, 7, 7])
model.branch1.5.conv.bias	cuda:0	torch.Size([8])
model.branch2.0.conv.weight	cuda:0	torch.Size([20, 3, 7, 7])
model.branch2.0.conv.bias	cuda:0	torch.Size([20])
model.branch2.2.conv.weight	cuda:0	torch.Size([40, 20, 5, 5])
model.branch2.2.conv.bias	cuda:0	torch.Size([40])
model.branch2.4.conv.weight	cuda:0	torch.Size([20, 40, 5, 5])
model.branch2.4.conv.bias	cuda:0	torch.Size([20])
model.branch2.5.conv.weight	cuda:0	torch.Size([10, 20, 5, 5])
model.branch2.5.conv.bias	cuda:0	torch.Size([10])
model.branch3.0.conv.weight	cuda:0	torch.Size([24, 3, 5, 5])
model.branch3.0.conv.b

In [9]:
# Change model weights tensors to be cuda tensors if is_cuda is true and cuda is available

In [10]:
#training configuration

disp_interval = 10

train_loss = 0
#step_cnt = 0
re_cnt = False
t = Timer()
t.tic()
# Set initial values
best_mae, best_mse, best_mape, best_epoch = 999999, 999999, 999999,0
best_mae_path = ''
best_mse_path = ''
best_mape_path = ''



In [11]:
# Hyperparameters

learning_rate = 0.00001
epochs = 100

# construct an optimizer

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr=learning_rate)

In [12]:
# Load the images, take note the num_pool argument

data_loader = ImageDataLoader(train_path, shuffle=False, pre_load=False, size = 200)
data_loader_val = ImageDataLoader(val_path, shuffle=False, pre_load=False, size = 40)

print('Training instances: {}'.format(data_loader.get_num_samples()))
print('Validation instances: {}'.format(data_loader_val.get_num_samples()))

Training instances: 200
Validation instances: 40


In [13]:
#current_date = datetime.now()
#cd_string = current_date.strftime("%d/%m/%Y %H:%M:%S")

#Tensorboard  config
use_tensorboard = True

writer = SummaryWriter(f'../output/tensorboard/runs/{learning_rate}_{epochs}_rerun')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [14]:
for epoch in range(1, epochs+1):    
    train_loss = 0.0
    model.train()
    for id, blob in enumerate(data_loader, 1):  
        im_data = blob['data']
        gt_data = blob['gt_density']
        metadata = blob['metadata']

        # Forward pass + backward pass + optimise
        try:
            density_map = model(im_data, gt_data)
            loss = model.loss
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 5)
            optimizer.step()

            # Write to tensorboard
            train_loss += loss.item()

            if id % disp_interval == 0:            
                gt_count = torch.sum(gt_data)                
                et_count = torch.sum(density_map)
                #utils.save_results(im_data,gt_data,density_map, output_dir)
                log_text = '[%3d, %3d] Image: %s, Actual Count: %6d, Estimated Count: %6.3f' % (epoch, id, metadata['img_path'], gt_count, et_count)
                log_print(log_text, color='green', attrs=['bold'])

        except Exception as e:
            #print(e)
            #print('continuing... image path: {}'.format(metadata['img_path']))
            continue

    # Overwrite the current model weights
    current_model = f'{method}_{learning_rate}.h5'
    save_name = os.path.join(output_dir, current_model)
    network.save_net(save_name, model)
            
    # Evaluate the mae and mse results by doing a forward pass against the validation dataset i.e data_loader_val for
    # each epoch
    MAEcrowddensity, MSEcrowddensity, MAPEcrowddensity, MAEweather, MSEweather, MAPEweather, MAE, MSE, RMSE, MAPE = evaluate_model(save_name, data_loader_val, is_cuda=is_cuda)
    
    # Pocket algorithm: Check to see if the current epoch mae is better than the best recorded one,
    # If it is, then overwrite the current best .h5 weights file
    if MAE < best_mae:
        # Save the new best mae and mse
        best_mae = MAE
        best_epoch = epoch
        best_model = f'best_MAE_epoch_{epoch}_{method}_{learning_rate}.h5'

        # Overwrite or create a new file for the best model for this learning rate
        save_name = os.path.join(output_dir, best_model)
        best_mae_path = save_name
        network.save_net(save_name, model)
        
    if MSE < best_mse:
        # Save the new best mae and mse
        best_mse = MSE
        best_epoch = epoch
        best_model = f'best_MSE_epoch_{epoch}_{method}_{learning_rate}.h5'

        # Overwrite or create a new file for the best model for this learning rate
        save_name = os.path.join(output_dir, best_model)
        best_mse_path = save_name
        network.save_net(save_name, model)
        
    if MAPE < best_mape:
        # Save the new best mae and mse
        best_mape = MAPE
        best_epoch = epoch
        best_model = f'best_MAPE_epoch_{epoch}_{method}_{learning_rate}.h5'

        # Overwrite or create a new file for the best model for this learning rate
        save_name = os.path.join(output_dir, best_model)
        best_mape_path = save_name
        network.save_net(save_name, model)
        
    # Print out the best epoch that beat the current best mae   
    log_text = 'EPOCH: %4d, Val MAE: %.3f, Val MSE: %.3f, Val RMSE: %.3f, Val MAPE: %.3f' % (epoch, MAE, MSE, RMSE, MAPE)
    log_print(log_text, color='blue', attrs=['bold'])

    # Save the results to tensorboard for each epoch
    if use_tensorboard:
        
        # overall segment
        writer.add_scalar("Overall/Val MAE", MAE, epoch)
        writer.add_scalar("Overall/Val MSE", MSE, epoch)
        writer.add_scalar("Overall/Val RMSE", RMSE, epoch)
        writer.add_scalar("Overall/Val MAPE", MAPE, epoch)
        writer.add_scalar("Overall/Train Loss", train_loss / data_loader.get_num_samples(), epoch)
        
        # crowd density segment
        writer.add_scalar('Crowd Density/High/MAE', MAEcrowddensity['High'], epoch)
        writer.add_scalar('Crowd Density/High/MSE', MSEcrowddensity['High'], epoch)
        writer.add_scalar('Crowd Density/High/RMSE', np.sqrt(MSEcrowddensity['High']), epoch)
        writer.add_scalar('Crowd Density/High/MAPE', MAPEcrowddensity['High'], epoch)
        
        writer.add_scalar('Crowd Density/Med/MAE', MAEcrowddensity['Med'], epoch)
        writer.add_scalar('Crowd Density/Med/MSE', MSEcrowddensity['Med'], epoch)
        writer.add_scalar('Crowd Density/Med/RMSE', np.sqrt(MSEcrowddensity['Med']), epoch)
        writer.add_scalar('Crowd Density/Med/MAPE', MAPEcrowddensity['Med'], epoch)
        
        writer.add_scalar('Crowd Density/Low/MAE', MAEcrowddensity['Low'], epoch)
        writer.add_scalar('Crowd Density/Low/MSE', MSEcrowddensity['Low'], epoch)
        writer.add_scalar('Crowd Density/Low/RMSE', np.sqrt(MSEcrowddensity['Low']), epoch)
        writer.add_scalar('Crowd Density/Low/MAPE', MAPEcrowddensity['Low'], epoch)
        
        # weather segment
        writer.add_scalar('Weather/No Degradation/MAE', MAEweather['None'], epoch)
        writer.add_scalar('Weather/No Degradation/MSE', MSEweather['None'], epoch)
        writer.add_scalar('Weather/No Degradation/RMSE', np.sqrt(MSEweather['None']), epoch)
        writer.add_scalar('Weather/No Degradation/MAPE', MAPEweather['None'], epoch)
        
        writer.add_scalar('Weather/Fog/MAE', MAEweather['Fog'], epoch)
        writer.add_scalar('Weather/Fog/MSE', MSEweather['Fog'], epoch)
        writer.add_scalar('Weather/Fog/RMSE', np.sqrt(MSEweather['Fog']), epoch)
        writer.add_scalar('Weather/Fog/MAPE', MAPEweather['Fog'], epoch)
        
        writer.add_scalar('Weather/Rain/MAE', MAEweather['Rain'], epoch)
        writer.add_scalar('Weather/Rain/MSE', MSEweather['Rain'], epoch)
        writer.add_scalar('Weather/Rain/RMSE', np.sqrt(MSEweather['Rain']), epoch)
        writer.add_scalar('Weather/Rain/MAPE', MAPEweather['Rain'], epoch)
        
        writer.add_scalar('Weather/Snow/MAE', MAEweather['Snow'], epoch)
        writer.add_scalar('Weather/Snow/MSE', MSEweather['Snow'], epoch)
        writer.add_scalar('Weather/Snow/RMSE', np.sqrt(MSEweather['Snow']), epoch)
        writer.add_scalar('Weather/Snow/MAPE', MAPEweather['Snow'], epoch)

[  1,  10] Image: 0771, Actual Count:     21, Estimated Count:  2.763
[  1,  20] Image: 0474, Actual Count:     50, Estimated Count: 34.859
[  1,  30] Image: 0784, Actual Count:   1643, Estimated Count: 53.968
[  1,  40] Image: 0373, Actual Count:     25, Estimated Count: 13.429
[  1,  50] Image: 0302, Actual Count:     89, Estimated Count: 23.526
[  1,  60] Image: 0845, Actual Count:   1555, Estimated Count: 63.467
[  1,  70] Image: 0075, Actual Count:    295, Estimated Count: 147.093
[  1,  80] Image: 0037, Actual Count:     74, Estimated Count: 51.742
[  1,  90] Image: 0428, Actual Count:     63, Estimated Count: 43.692
[  1, 100] Image: 0920, Actual Count:     34, Estimated Count: 31.696
[  1, 110] Image: 0298, Actual Count:     33, Estimated Count: 97.078
[  1, 120] Image: 0980, Actual Count:    530, Estimated Count: 81.097
[  1, 130] Image: 0752, Actual Count:    421, Estimated Count: 1317.062
[  1, 140] Image: 0765, Actual Count:   4323, Estimated Count: 342.991
[  1, 150] Image

/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
../models/mcnnpytorch/src/network.py:64: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  v = Variable(torch.as_tensor(x).type(dtype), requires_grad = False, volatile = True)


EPOCH:    1, Val MAE: 334.623, Val MSE: 653574.295, Val RMSE: 808.439, Val MAPE: 2.069
[  2,  10] Image: 0771, Actual Count:     21, Estimated Count: 44.814
[  2,  20] Image: 0474, Actual Count:     50, Estimated Count: 248.599
[  2,  30] Image: 0784, Actual Count:   1643, Estimated Count: 286.923
[  2,  40] Image: 0373, Actual Count:     25, Estimated Count: 51.486
[  2,  50] Image: 0302, Actual Count:     89, Estimated Count: 70.116
[  2,  60] Image: 0845, Actual Count:   1555, Estimated Count: 165.592
[  2,  70] Image: 0075, Actual Count:    295, Estimated Count: 300.520
[  2,  80] Image: 0037, Actual Count:     74, Estimated Count: 86.917
[  2,  90] Image: 0428, Actual Count:     63, Estimated Count: 52.127
[  2, 100] Image: 0920, Actual Count:     34, Estimated Count: 41.927
[  2, 110] Image: 0298, Actual Count:     33, Estimated Count: 110.764
[  2, 120] Image: 0980, Actual Count:    530, Estimated Count: 100.778
[  2, 130] Image: 0752, Actual Count:    421, Estimated Count: 1843

[  6, 170] Image: 0082, Actual Count:     11, Estimated Count: 2081.967
[  6, 180] Image: 0892, Actual Count:    523, Estimated Count: 71.555
[  6, 190] Image: 0685, Actual Count:    408, Estimated Count: 573.851
EPOCH:    6, Val MAE: 342.998, Val MSE: 454720.907, Val RMSE: 674.330, Val MAPE: 2.691
[  7,  10] Image: 0771, Actual Count:     21, Estimated Count: 109.765
[  7,  20] Image: 0474, Actual Count:     50, Estimated Count: 127.674
[  7,  30] Image: 0784, Actual Count:   1643, Estimated Count: 513.532
[  7,  40] Image: 0373, Actual Count:     25, Estimated Count: 27.336
[  7,  50] Image: 0302, Actual Count:     89, Estimated Count: 57.278
[  7,  60] Image: 0845, Actual Count:   1555, Estimated Count: 481.490
[  7,  70] Image: 0075, Actual Count:    295, Estimated Count: 412.028
[  7,  80] Image: 0037, Actual Count:     74, Estimated Count: 163.853
[  7,  90] Image: 0428, Actual Count:     63, Estimated Count: 22.817
[  7, 100] Image: 0920, Actual Count:     34, Estimated Count: 7

[ 11, 140] Image: 0765, Actual Count:   4323, Estimated Count: 1190.038
[ 11, 150] Image: 0261, Actual Count:     68, Estimated Count: 26.382
[ 11, 160] Image: 0322, Actual Count:     22, Estimated Count: 149.409
[ 11, 170] Image: 0082, Actual Count:     11, Estimated Count: 2286.135
[ 11, 180] Image: 0892, Actual Count:    523, Estimated Count: 72.928
[ 11, 190] Image: 0685, Actual Count:    408, Estimated Count: 631.966
EPOCH:   11, Val MAE: 305.168, Val MSE: 380946.473, Val RMSE: 617.209, Val MAPE: 2.169
[ 12,  10] Image: 0771, Actual Count:     21, Estimated Count: 95.694
[ 12,  20] Image: 0474, Actual Count:     50, Estimated Count: 77.216
[ 12,  30] Image: 0784, Actual Count:   1643, Estimated Count: 553.448
[ 12,  40] Image: 0373, Actual Count:     25, Estimated Count: 22.288
[ 12,  50] Image: 0302, Actual Count:     89, Estimated Count: 36.650
[ 12,  60] Image: 0845, Actual Count:   1555, Estimated Count: 520.757
[ 12,  70] Image: 0075, Actual Count:    295, Estimated Count: 40

...

[ 79, 140] Image: 0765, Actual Count:   4323, Estimated Count: 1069.157
[ 79, 150] Image: 0261, Actual Count:     68, Estimated Count: 42.182
[ 79, 160] Image: 0322, Actual Count:     22, Estimated Count: 85.450
[ 79, 170] Image: 0082, Actual Count:     11, Estimated Count: 2701.785
[ 79, 180] Image: 0892, Actual Count:    523, Estimated Count: 106.217
[ 79, 190] Image: 0685, Actual Count:    408, Estimated Count: 503.444
EPOCH:   79, Val MAE: 244.176, Val MSE: 177759.021, Val RMSE: 421.615, Val MAPE: 1.801
[ 80,  10] Image: 0771, Actual Count:     21, Estimated Count: 12.436
[ 80,  20] Image: 0474, Actual Count:     50, Estimated Count: 13.915
[ 80,  30] Image: 0784, Actual Count:   1643, Estimated Count: 400.429
[ 80,  40] Image: 0373, Actual Count:     25, Estimated Count: 16.134
[ 80,  50] Image: 0302, Actual Count:     89, Estimated Count: 17.370
[ 80,  60] Image: 0845, Actual Count:   1555, Estimated Count: 676.224
[ 80,  70] Image: 0075, Actual Count:    295, Estimated Count: 26

[ 84, 110] Image: 0298, Actual Count:     33, Estimated Count: 30.180
[ 84, 120] Image: 0980, Actual Count:    530, Estimated Count: 219.149
[ 84, 130] Image: 0752, Actual Count:    421, Estimated Count: 209.760
[ 84, 140] Image: 0765, Actual Count:   4323, Estimated Count: 1046.362
[ 84, 150] Image: 0261, Actual Count:     68, Estimated Count: 42.971
[ 84, 160] Image: 0322, Actual Count:     22, Estimated Count: 82.715
[ 84, 170] Image: 0082, Actual Count:     11, Estimated Count: 2688.934
[ 84, 180] Image: 0892, Actual Count:    523, Estimated Count: 107.460
[ 84, 190] Image: 0685, Actual Count:    408, Estimated Count: 490.822
EPOCH:   84, Val MAE: 238.910, Val MSE: 168038.221, Val RMSE: 409.925, Val MAPE: 1.767
[ 85,  10] Image: 0771, Actual Count:     21, Estimated Count: 12.764
[ 85,  20] Image: 0474, Actual Count:     50, Estimated Count: 14.956
[ 85,  30] Image: 0784, Actual Count:   1643, Estimated Count: 401.164
[ 85,  40] Image: 0373, Actual Count:     25, Estimated Count: 1

[ 89,  80] Image: 0037, Actual Count:     74, Estimated Count: 146.222
[ 89,  90] Image: 0428, Actual Count:     63, Estimated Count: 22.224
[ 89, 100] Image: 0920, Actual Count:     34, Estimated Count: 48.490
[ 89, 110] Image: 0298, Actual Count:     33, Estimated Count: 30.941
[ 89, 120] Image: 0980, Actual Count:    530, Estimated Count: 223.458
[ 89, 130] Image: 0752, Actual Count:    421, Estimated Count: 208.610
[ 89, 140] Image: 0765, Actual Count:   4323, Estimated Count: 1035.176
[ 89, 150] Image: 0261, Actual Count:     68, Estimated Count: 43.827
[ 89, 160] Image: 0322, Actual Count:     22, Estimated Count: 79.785
[ 89, 170] Image: 0082, Actual Count:     11, Estimated Count: 2682.231
[ 89, 180] Image: 0892, Actual Count:    523, Estimated Count: 110.041
[ 89, 190] Image: 0685, Actual Count:    408, Estimated Count: 492.591
EPOCH:   89, Val MAE: 234.128, Val MSE: 158978.454, Val RMSE: 398.721, Val MAPE: 1.727
[ 90,  10] Image: 0771, Actual Count:     21, Estimated Count: 1

[ 94,  50] Image: 0302, Actual Count:     89, Estimated Count: 18.424
[ 94,  60] Image: 0845, Actual Count:   1555, Estimated Count: 691.947
[ 94,  70] Image: 0075, Actual Count:    295, Estimated Count: 263.944
[ 94,  80] Image: 0037, Actual Count:     74, Estimated Count: 148.966
[ 94,  90] Image: 0428, Actual Count:     63, Estimated Count: 23.242
[ 94, 100] Image: 0920, Actual Count:     34, Estimated Count: 47.235
[ 94, 110] Image: 0298, Actual Count:     33, Estimated Count: 30.742
[ 94, 120] Image: 0980, Actual Count:    530, Estimated Count: 223.666
[ 94, 130] Image: 0752, Actual Count:    421, Estimated Count: 206.174
[ 94, 140] Image: 0765, Actual Count:   4323, Estimated Count: 1019.653
[ 94, 150] Image: 0261, Actual Count:     68, Estimated Count: 44.420
[ 94, 160] Image: 0322, Actual Count:     22, Estimated Count: 76.488
[ 94, 170] Image: 0082, Actual Count:     11, Estimated Count: 2644.847
[ 94, 180] Image: 0892, Actual Count:    523, Estimated Count: 112.395
[ 94, 190]

[ 99,  20] Image: 0474, Actual Count:     50, Estimated Count: 13.811
[ 99,  30] Image: 0784, Actual Count:   1643, Estimated Count: 393.314
[ 99,  40] Image: 0373, Actual Count:     25, Estimated Count: 13.366
[ 99,  50] Image: 0302, Actual Count:     89, Estimated Count: 17.912
[ 99,  60] Image: 0845, Actual Count:   1555, Estimated Count: 691.693
[ 99,  70] Image: 0075, Actual Count:    295, Estimated Count: 264.863
[ 99,  80] Image: 0037, Actual Count:     74, Estimated Count: 152.458
[ 99,  90] Image: 0428, Actual Count:     63, Estimated Count: 24.086
[ 99, 100] Image: 0920, Actual Count:     34, Estimated Count: 45.602
[ 99, 110] Image: 0298, Actual Count:     33, Estimated Count: 29.059
[ 99, 120] Image: 0980, Actual Count:    530, Estimated Count: 221.213
[ 99, 130] Image: 0752, Actual Count:    421, Estimated Count: 200.900
[ 99, 140] Image: 0765, Actual Count:   4323, Estimated Count: 1013.316
[ 99, 150] Image: 0261, Actual Count:     68, Estimated Count: 45.172
[ 99, 160] I

In [17]:
log_text = f'BEST MAE: {best_mae}, BEST MSE: {best_mse}, BEST MAPE: {best_mape}'
log_print(log_text, color='green', attrs=['bold'])

BEST MAE: 220.8581748187542, BEST MSE: 138140.83970361564, BEST MAPE: 1.6216285839890578


In [16]:
evaluate_model(best_mape_path, data_loader_val, is_cuda=is_cuda)

({'High': 675.8092803955078,
  'Med': 206.29848787058955,
  'Low': 59.20128081242243},
 {'High': 691832.3010932595, 'Med': 86554.06327538284, 'Low': 6310.7189453769},
 {'High': 0.44786629213326457,
  'Med': 1.4180943820490783,
  'Low': 2.5008034259805982},
 {'None': 230.39322600762048,
  'Fog': 10.947092056274414,
  'Rain': 200.7983341217041,
  'Snow': 127.62709617614746},
 {'None': 150500.0121030571,
  'Fog': 119.83882448854638,
  'Rain': 45612.31896586356,
  'Snow': 16288.675678355594},
 {'None': 1.7032147619928493,
  'Fog': 0.8420839425538912,
  'Rain': 0.9135706964798094,
  'Snow': 0.8801865923062263},
 220.8581748187542,
 138140.83970361564,
 371.67302794743614,
 1.6216285839890578)

In [40]:
is_cuda = False
model2 = CrowdCounter(is_cuda=is_cuda)
network.weights_normal_init(model2, dev=0.01)

from models.csrnet_pytorch.src.network import load_net
load_net('../output/mcnn/saved_models/JHU/rerun/best_MSE_epoch_100_mcnn_1e-05.h5', model2)
model2.eval()

# Test data path
test_path = '../data/JHU/test/consolidated'
test_gt_path = '../data/JHU/test/gt'
data_loader_test = ImageDataLoader(test_path, shuffle=False, pre_load=False, size=40)

for i in [4,18,11,8,32,7]:
    test_image = data_loader_test.get_test_input(index=i)
    density_map_test = model2(test_image['data'])
    save_density_map(density_map_test.detach().numpy(), output_dir, fname=f'results_{i}.jpg')
    
    im_gray = cv2.imread(os.path.join(output_dir, f'results_{i}.jpg'), cv2.IMREAD_GRAYSCALE)
    im_jet = cv2.applyColorMap(im_gray, cv2.COLORMAP_JET)
    cv2.imwrite(os.path.join(output_dir, f'results_{i}.jpg'), im_jet)

In [39]:
data_loader_test.get_test_input(index=8)

{'data': tensor([[[[0.4118, 0.4078, 0.4078,  ..., 0.7098, 0.7176, 0.7020],
           [0.4157, 0.4118, 0.4118,  ..., 0.7255, 0.7412, 0.7294],
           [0.3490, 0.3529, 0.3765,  ..., 0.6863, 0.7059, 0.6941],
           ...,
           [0.7961, 0.7725, 0.7922,  ..., 0.6000, 0.6863, 0.6706],
           [0.7882, 0.7608, 0.7882,  ..., 0.6118, 0.6627, 0.6980],
           [0.7804, 0.7529, 0.7804,  ..., 0.6784, 0.6627, 0.6784]],
 
          [[0.4392, 0.4353, 0.4353,  ..., 0.7216, 0.7294, 0.7137],
           [0.4431, 0.4392, 0.4392,  ..., 0.7373, 0.7529, 0.7412],
           [0.3765, 0.3804, 0.4039,  ..., 0.6980, 0.7176, 0.7059],
           ...,
           [0.7882, 0.7647, 0.7843,  ..., 0.4902, 0.5725, 0.5569],
           [0.7804, 0.7529, 0.7804,  ..., 0.4980, 0.5373, 0.5725],
           [0.7725, 0.7451, 0.7725,  ..., 0.5647, 0.5373, 0.5490]],
 
          [[0.4118, 0.4078, 0.4078,  ..., 0.6941, 0.7020, 0.6863],
           [0.4157, 0.4118, 0.4118,  ..., 0.7098, 0.7255, 0.7137],
           [0.34

In [56]:
test_image_32 = data_loader_test.get_test_input(index=32)
density_map_32 = model(test_image_32['data'])
save_density_map(density_map_32.detach().numpy(), output_dir, fname=f'results_32.jpg')
# test_image_18
im_gray32 = cv2.imread(os.path.join(output_dir, 'results_32.jpg'), cv2.IMREAD_GRAYSCALE)
im_jet32 = cv2.applyColorMap(im_gray32, cv2.COLORMAP_JET)
cv2.imwrite(os.path.join(output_dir, 'results_32_colour.jpg'), im_jet32)

True

In [18]:
import cv2
im_gray0 = cv2.imread(os.path.join(output_dir, 'results_0.jpg'), cv2.IMREAD_GRAYSCALE)
im_jet0 = cv2.applyColorMap(im_gray0, cv2.COLORMAP_JET)
im_gray1 = cv2.imread(os.path.join(output_dir, 'results_1.jpg'), cv2.IMREAD_GRAYSCALE)
im_jet1 = cv2.applyColorMap(im_gray1, cv2.COLORMAP_JET)
im_gray2 = cv2.imread(os.path.join(output_dir, 'results_2.jpg'), cv2.IMREAD_GRAYSCALE)
im_jet2 = cv2.applyColorMap(im_gray2, cv2.COLORMAP_JET)
im_gray39 = cv2.imread(os.path.join(output_dir, 'results_39.jpg'), cv2.IMREAD_GRAYSCALE)
im_jet39 = cv2.applyColorMap(im_gray39, cv2.COLORMAP_JET)

cv2.imwrite(os.path.join(output_dir, 'results_0_colour.jpg'), im_jet0)
cv2.imwrite(os.path.join(output_dir, 'results_1_colour.jpg'), im_jet1)
cv2.imwrite(os.path.join(output_dir, 'results_2_colour.jpg'), im_jet2)
cv2.imwrite(os.path.join(output_dir, 'results_39_colour.jpg'), im_jet39)

True